## Teste de hipótese para problemas de negócio.

Área: Telecon.

In [31]:
# Preparação do ambiente
import pandas as pd
import numpy as np
from scipy.stats import stats
from scipy.stats import chi2_contingency
from platform import python_version
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Versão da interpretador e pacotes utilizados
#!pip install -q -U watermark
%load_ext watermark
%watermark -a "Fagner F. Farias" --iv
print(f"Versão Python:{python_version()}")

Author: Fagner F. Farias

scipy : 1.10.1
numpy : 1.24.3
pandas: 1.5.3

Versão Python:3.11.3


In [3]:
# Carga dos dados 
dados = pd.read_csv("dataset.csv")

In [4]:
# Visualizando
dados.head()

,id_cliente,genero,canal_atendimento,regiao,estado_civil,segmento,consumo_medio_mensal_antes_upgrade,consumo_medio_mes_anterior_ao_upgrade,consumo_medio_primeiro_mes_apos_upgrade,consumo_medio_segundo_mes_apos_upgrade
0,70,0,4,1,1,1,57,49.2,52,57.2
1,121,1,4,2,1,3,68,63.6,59,64.9
2,86,0,4,3,1,1,44,64.8,33,36.3
3,141,0,4,3,1,3,63,56.4,44,48.4
4,172,0,4,2,1,2,47,68.4,52,57.2


In [15]:
# Metadados
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 10 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   id_cliente                               200 non-null    int64  
 1   genero                                   200 non-null    int64  
 2   canal_atendimento                        200 non-null    int64  
 3   regiao                                   200 non-null    int64  
 4   estado_civil                             200 non-null    int64  
 5   segmento                                 200 non-null    int64  
 6   consumo_medio_mensal_antes_upgrade       200 non-null    int64  
 7   consumo_medio_mes_anterior_ao_upgrade    200 non-null    float64
 8   consumo_medio_primeiro_mes_apos_upgrade  200 non-null    int64  
 9   consumo_medio_segundo_mes_apos_upgrade   200 non-null    float64
dtypes: float64(2), int64(8)
memory usage: 15.8 KB


O objetivo desta análise é demostrar a aplicação do testes de hipóteses estatísticos para resolver problemas de negócio. Portanto, não se dará atenção às demais atividades envolvendo um projetos de dados, como DataWrangling, Pré-processamento, etc.

Assim, verificando que nãoh há dados nulos, passamos aos teste de hipóteses.

#### **Problemas de negócio**

**1 - O consumo médio de largura de banda do mês anterior do upgrade, foi maior que 50?**

**Modelagem do teste de hipótese**

Teste: test-t de 1 amostra (de calda superior)

>H0: O consumo de banda larga no mês anterior foi ou menor que 50.

>Ha: O consumo de banda larga no mês atenrior foi maior de 50.


In [13]:
# Aplicando teste
statistic, pvalue = stats.ttest_1samp(a = dados.consumo_medio_mensal_antes_upgrade,
                           popmean = 50,
                           alternative = 'greater')

print(f'O retorno foi p-value de: {pvalue}. \nPortanto, rejeita-se a H0, concluindo-se que há evidências estatísticamente relevantes \nde que o consumo foi maior que 50.')

O retorno foi p-value de: 0.0011969244515674076. 
Portanto, rejeita-se a H0, concluindo-se que há evidências estatísticamente relevantes 
de que o consumo foi maior que 50.


**2 - Houve diferença no consumo de largura de banda antes e depois do pugrade, considerando o primeiro mês, após o upgrande?**

**Modelagem do teste de hipótese**

Teste: test-t de 2 amostras relacionadas.

>H0: O consumo de banda larga foi igual antes do upgrade, e no mês seguinte à implementação.

>Ha: O consumo de banda larga não foi igual antes do upgrade, e no mês seguinte à implementação.


In [15]:
# Aplicando teste
statistic, pvalue = stats.ttest_rel(a = dados.consumo_medio_mensal_antes_upgrade,
                                   b = dados.consumo_medio_primeiro_mes_apos_upgrade)

print(f'O pvalor foi de {pvalue}. \nPortanto, não há evidências estatísticas para rejeitar a H0.\nAssim, tem-se que o consumo foi igual, antes e depois do upgrade.')

O pvalor foi de 0.3868186820914985. 
Portanto, não há evidências estatísticas para rejeitar a H0.
Assim, tem-se que o consumo foi igual, antes e depois do upgrade.


**3 - O gênero do cliente influenciou o consumo de banda, após o upgrade?**

**Modelagem do teste de hipótese**

Teste: test-t de 2 amostras independentes.

>H0: O consumo de banda larga não foi igual entre os sexos feminino e masculino.

>Ha: O consumo de banda larga foi igual entre os sexos feminino e masculino.


In [21]:
# Extração das amostras
masculino = dados.consumo_medio_primeiro_mes_apos_upgrade[dados.genero == 0]
feminino = dados.consumo_medio_primeiro_mes_apos_upgrade[dados.genero == 1]

In [26]:
# Avaliando suposição - igualdade de variância
    #H0 - as amostas possuem a mesma variância
    #Ha - as amostras tem variância diferentes

statistics, pvalue = stats.f_oneway(masculino,
                                    feminino)

print(f'O pvalue é de {pvalue}, portanto, há evidências estatísticas para rejeitar a H0. \nConcluin-se que as amostras tem variâncias diferestes.')

O pvalue é de 0.0002462546120354903, portanto, há evidências estatísticas para rejeitar a H0. 
Concluin-se que as amostras tem variâncias diferestes.


In [29]:
# Aplicando teste
statistic, pvalue = stats.ttest_ind(a = masculino,
                                   b = feminino,
                                   equal_var= False) #informando que as variâncias são diferentes

print(f'O pvalue foi de {pvalue}.\nPortanto, há evidências estatísticas para rejeitar a H0, assumindo-se que o consumo foi igual para ambos os sexos.')

O pvalue foi de 0.000340884935942662.
Portanto, há evidências estatísticas para rejeitar a H0, assumindo-se que o consumo foi igual para ambos os sexos.


**3 - Há relação entre região e segmento do cliente?**

**Modelagem do teste de hipótese**

Teste: test-t chi-quadrado (para variáveis categóricas).

>H0: Não há relacionamento entre as região e segmento do cliente.

>Ha: Há relacionamento entre as região e segmento do cliente.


In [30]:
# Confeccionando tabela de contingência

tabelaContingencia = pd.crosstab(dados.regiao,
                                dados.segmento,
                                margins=True)

tabelaContingencia

segmento,1,2,3,All
regiao,,,,
1,16,19,12,47
2,20,44,31,95
3,9,42,7,58
All,45,105,50,200


In [34]:
# Aplicando teste

chi, p, dof, ex = chi2_contingency(observed = tabelaContingencia)

print(f'O pvalue foi de {p}.\nNeste caso, as duas hipóteses merecem ser analisadas mais de perto, haja vis que, \n o pvalue ficou pouco acima do nível de significância desejado.')

O pvalue foi de 0.055282939487992254.
Neste caso, as duas hipóteses merecem ser analisadas mais de perto, haja vis que, 
 o pvalue ficou pouco acima do nível de significância desejado.


As questões acima são exemplos de quetões que podem surgir no dia-a-dia, e, decididas de forma objetiva com base em dados.